# Домашнее задание № 3. Исправление опечаток

In [1]:
from collections import Counter
import re

bad = open('data/sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('data/correct_sents.txt', encoding='utf8').read().splitlines()
corpus = open('data/wiki_data.txt', encoding='utf8').read()

In [2]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

vocab = Counter(re.findall('\w+', corpus.lower()))

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', ngram_range=(1,3), max_features=1000)
X = vec.fit_transform(vocab)

### 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [44]:
import textdistance

def get_closest_match_with_metric(text, lookup, topn=20, metric=textdistance.levenshtein):
    similarities = Counter()
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word)        
    return similarities.most_common(topn)

def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn * 4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)

    top_dist = closest[0][1]
    top_words = []
    for item in closest:
        if item[1] == top_dist:
            top_words.append(item[0])
    most_prob = dict()
    for word in top_words:
        most_prob[word] = P(word)
    most_common = max(most_prob, key=most_prob.get)
    if len(top_words) > 1:
        return most_common
    else:
        return top_words[0]

N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1

In [45]:
from string import punctuation

def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

def spell_checking(topn=3, vec=vec, metric=textdistance.damerau_levenshtein):
    mistakes = []
    total_mistaken = 0
    mistaken_fixed = 0

    total_correct = 0
    correct_broken = 0

    total = 0
    correct = 0

    cashed = {}
    for i in range(len(true)):
        word_pairs = align_words(true[i], bad[i])
        for pair in word_pairs:
            if predict_mistaken(pair[1], vocab):
                pred = cashed.get(pair[1], get_closest_hybrid_match(pair[1], X, vec, topn=topn, metric=metric))
                cashed[pair[1]] = pred
            else:
                pred = pair[1]


            if pred == pair[0]:
                correct += 1
            else:
                mistakes.append((pair[0], pair[1], pred))
            total += 1

            if pair[0] == pair[1]:
                total_correct += 1
                if pair[0] != pred:
                    correct_broken += 1
            else:
                total_mistaken += 1
                if pair[0] == pred:
                    mistaken_fixed += 1

        if not i % 100:
            print(i)
    print(correct/total)
    print(mistaken_fixed/total_mistaken)
    print(correct_broken/total_correct)

In [37]:
spell_checking()

0
100
200
300
400
500
600
700
800
900
0.8556278139069535
0.48835403726708076
0.09004249454461927


Удалось немного повысить точность

## 2.  Symspell (5 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Там к словам применяется только одна операция - удаление символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово   
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [21]:
corpus = open('data/wiki_data.txt', encoding='utf8').read()
bad = open('data/sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('data/correct_sents.txt', encoding='utf8').read().splitlines()

vocab = Counter(re.findall('\w+', corpus.lower()))

In [22]:
correct_words = list(vocab.keys())

In [23]:
%%time
del_dict = dict()
for w in correct_words:
    for i in range(len(w)):
        w_with_del = w[0:i] + w[i + 1:]
        if w_with_del in del_dict.keys():
            del_dict[w_with_del] = del_dict[w_with_del] + [w]
        else:
            del_dict[w_with_del] = [w]

CPU times: user 8.62 s, sys: 538 ms, total: 9.16 s
Wall time: 10.2 s


In [24]:
from functools import lru_cache

@lru_cache()
def get_all_dels(w):
    dels_lst = []
    for i in range(len(w)):
        w_with_del = w[0:i] + w[i + 1:]
        dels_lst.append(w_with_del)
    return dels_lst

@lru_cache()
def get_candidate(w):
    cand_p = dict()
    dels_lst = get_all_dels(w)
    for d in dels_lst:
        if d in del_dict.keys():
            for el in del_dict[d]:
                cand_p[el] = P(el)
    if not cand_p:
        return w
    return max(cand_p, key=cand_p.get)

In [32]:
from string import punctuation

l = 0

def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

mistakes = []

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab):
            predicted = cashed.get(pair[1], get_candidate(pair[1]))
            cashed[pair[1]] = predicted
        else:
            predicted = pair[1]
        
        
        if predicted == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], predicted))
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1
        
    if not i % 100:
        print(i)

0
100
200
300
400
500
600
700
800
900


In [33]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8560280140070035
0.19953416149068323
0.04685884920179166


## *3. Настройка гиперпараметров. (2 балла)

У метода из первого заданий много гиперпараметров (те которые нужно подбирать самостоятельно). Это параметры векторайзера, topn, метрика редактирования. Поэкспериментируйте с ними. 

Проведите как минимум 10 экспериментов с разными параметрами. Для каждого эксперимента укажите мотивацию (например, "слишком маленький topn в get_closest_match_vec приводит к тому, что некоторые хорошие варианты не доходят до get_closest_match_with_metric, попробуем его увеличить")

Старайтесь получить улучшение, но если улучшить не получится, то это не страшно. Главное, чтобы эксперименты были осмысленными, а не рандомными. 